In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches
import matplotlib as mpl

### colors & groups

In [ ]:

cdr_method_colors = {
    
    'SCS': '#a47551', 
    
    'A/R': '#238b45', 
    'General forestry': '#66c2a4', 
    'Agroforestry': '#b2e2e2', 
    'Forest management': '#edf8fb',

    'Biochar': '#ce1256',
    'Restoration of landscapes and peats': '#df65b0',
    'EW': '#d7b5d8', 
  
    'OAE': '#016c59', 
    'OIF/AU': '#1c9099', 
    'DOC': '#67a9cf',
    'Algae ': '#bdc9e1', 
    'Blue carbon': '#f6eff7',
       
    'CCS': '#e31a1c',
    'BECCS': '#fd8d3c', 
    'DACCS': '#fecc5c',
    'CCUS': '#ffffb2', 
    
    'General CDR': '#525252', 
    'Other': '#969696',    
}

# Define the custom color scheme for each CDR group
cdr_group_colors = {
    "SCS": '#C1A28A' ,            # brown for SCS
    "Forestry": '#8fbc8f',         # green for Forestry
    "Terrestrial CDR": '#df65b0',  # green for Terrestrial CDR
 
    "Marine CDR": '#7eaed3',    # bluew for Marine CDR
    "CCS": '#ff7f00',            # orange for CCS
    "General CDR/Other": '#e5e5e5', # grey for General/Other
}  

mrv_topic_colors = {
    'Quantification': '#7da7b8',   # darker bluish-grey
    'Monitoring': '#8dc39f',        # stronger soft green
    'Quality': '#c76a85',           # berry red
    'Governance': '#b39bc8',        # stronger lavender
    'Reporting': '#e1a6b8',         # stronger soft pink
    'Verification': '#a8a8a8',      # medium grey
    'External impacts': '#d0c97b',   # stronger soft yellow
    'General MRV': '#e5b138',      # muted yellow
    
}

cdr_method_label = \
{
    'SCS': "Soil Carbon Squestration",
    'A/R': "Afforestation/Reforestation",
    'General forestry': "General Forestry",
    'Agroforestry': "Agroforestry",
    'Forest management': "Forest Management",
    'Biochar': "Biochar",
    'Restoration of landscapes and peats': "Restoration of \nLandscapes/Peats",
    'EW': "Enhanced Weathering \n(land-based)",
    'OAE': "Ocean Alkalinity \nEnhancement",
    'OIF/AU': "Ocean Iron Fertilization", 
    'DOC':"Direct Ocean \nCapture",
    'Algae ':"Algae",
    'Blue carbon': "Blue Carbon \nManagement",
    'CCS':"CCS",
    'BECCS':"BECCS",
    'DACCS':"DACCS",
    'CCUS':"CCUS",
    'General CDR':"General CDR",
    'Other':"Other"
}      

resMethod_colors = {
     'Modelling':'#e34a33',
     'Data analysis/Statistical analysis/Econometric':'#fdbb84',
     'Remote sensing':'#253494',
     'Experiment - Field study':'#2c7fb8',
     'Experiment - Laboratory experiment':'#41b6c4',
     'Qualitative research':'#31a354',
     'Review': '#a1d99b',
     'Unknown': '#969696',
}

In [ ]:
cdr_group_mapping = {
     # Marine CDR group
    "OIF/AU":"Marine CDR",
    "Blue carbon": "Marine CDR",
    "OAE": "Marine CDR",
    "DOC": "Marine CDR",
    "Algae ": "Marine CDR",
    
    # Terrestrial CDR group
    "EW": "Terrestrial CDR",
    "Biochar": "Terrestrial CDR",
    "Restoration of landscapes and peats": "Terrestrial CDR",
    
    # CCS group
    "DACCS": "CCS",
    "BECCS": "CCS",
    "CCUS": "CCS",
    "CCS": "CCS",
    
    # General CDR/Other group
    "General CDR": "General CDR/Other",
    "Other": "General CDR/Other",

    # SCS
    "SCS": "SCS",

    # Forestry
    "General forestry": "Forestry",
    "A/R": "Forestry",
    "Agroforestry": "Forestry",
    "Forest management": "Forestry",
}

### Get and clean data

In [ ]:
df = pd.read_excel("MRVdata_2210.xlsx") # older version: MRVdata_0810 ?
df_filtered = df[df['Year'] != 2024]
df_included = df_filtered[df_filtered['Inclusion/exclusion'] == 'Inclusion'].copy() # new dataframe defined to exclude studies without access or excluded papers

### additonal data

def extract_CDR_method_focus(row):
    CDR_method = []
    CDR_focus = []
    
    for column in CDR_method_colors.keys():  # previously defined list with columns that contain CDR methods
        if row[column] > 0:
            CDR_method.append(column)
            CDR_focus.append(row[column])
    
    return [CDR_method, CDR_focus]
# add new columns to the data frame
df_included['CDR_focus'] = df_included.apply(lambda row: extract_CDR_method_focus(row), axis=1)
df_included['CDR_method'] = df_included.CDR_focus.str[0]
df_included['CDR_focus'] = df_included.CDR_focus.str[1]


#add new column for MRV topic and study focus to the dataset
def extract_MRV_topic_focus(row):
    MRV_topic = []
    MRV_focus = []
    
    for column in mrv_topic_colors.keys():  # previously defined list with columns that contain MRV topics
        if row[column] > 0:
            MRV_topic.append(column)
            MRV_focus.append(row[column])
    
    return [MRV_topic, MRV_focus]
# add new columns to the data frame
df_included['MRV_focus'] = df_included.apply(lambda row: extract_MRV_topic_focus(row), axis=1)
df_included['MRV_topic'] = df_included.MRV_focus.str[0]
df_included['MRV_focus'] = df_included.MRV_focus.str[1]


### explode
# explode CDR method and focus
df_CDRexplode = df_included.explode(['CDR_method', 'CDR_focus'])

# explode MRV topic and focus
df_MRVexplode = df_included.explode(['MRV_topic', 'MRV_focus'])

### create larger groups
# Create a new column ‘CDR_group’ in df_exploded based on the ‘CDR_method’ column
df_CDRexplode['CDR_group'] = df_CDRexplode['CDR_method'].map(cdr_group_mapping)


#### CDR method and focus

In [ ]:
# --> to SI


# Get the counts of each CDR_focus per CDR_method
df_counts = df_exploded.groupby(['CDR_method', 'CDR_focus'], observed=False).size().unstack(fill_value=0)

# Plotting
fig, ax = plt.subplots(figsize=(8, 6))

# Stacked bar plot
df_counts.plot(kind='bar', stacked=True, ax=ax, color=['#87CEEB', '#6CA6CD', '#4A708B'])

# Customizing plot
ax.set_xlabel('CDR Method')
ax.set_ylabel('Count')
ax.set_title('Stacked Bar Plot of CDR Technologies by Focus')
plt.xticks(rotation=90)
legend_cdrfocus = ['Main focus', 'Side focus', 'Framing']
plt.legend(legend_cdrfocus, title='CDR Focus', bbox_to_anchor=(1.05, 1), loc='upper left')  # Legend outside the plot
plt.tight_layout()
plt.show()

#### CDR method - donut

In [ ]:
cdr_meth = df_CDRexplode.groupby("CDR_method", as_index=False).size()
cdr_meth["col"] = cdr_meth.CDR_method.replace(cdr_method_colors)
# sorting
cdr_meth.CDR_method = cdr_meth.CDR_method.astype("category")
cdr_meth.CDR_method = cdr_meth.CDR_method.cat.set_categories(cdr_method_colors.keys())
cdr_meth = cdr_meth.sort_values("CDR_method")

df_CDRexplode["CDR_group"] = df_CDRexplode.CDR_method.replace(cdr_group_mapping)
cdr_group = df_CDRexplode.groupby("CDR_group", as_index=False).size()
cdr_group["col"] = cdr_group.CDR_group.replace(cdr_group_colors)
# sorting
cdr_group.CDR_group = cdr_group.CDR_group.astype("category")
cdr_group.CDR_group = cdr_group.CDR_group.cat.set_categories(cdr_group_colors.keys())
cdr_group = cdr_group.sort_values("CDR_group")

In [ ]:
fig, ax = plt.subplots()

ax.pie(cdr_group["size"], colors=cdr_group["col"], startangle=90, radius=1.5)
my_circle = plt.Circle( (0,0), 1.1, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)

ax.pie(cdr_meth["size"], colors=cdr_meth["col"], startangle=90, radius=1)

legend_meth = []
legend_group = []
for c in cdr_method_colors.keys():
    legend_meth.append(mpatches.Patch(color=cdr_method_colors[c], label=cdr_method_label[c]))
for c in cdr_group_colors.keys():
    legend_group.append(mpatches.Patch(color=cdr_group_colors[c], label=c))    
leg_meth = plt.legend(handles=legend_meth, bbox_to_anchor=(1.5,-0.25), fontsize=10, frameon=False,ncol=2)
#leg_gro = plt.legend(handles=legend_group, bbox_to_anchor=(1,0), fontsize=10, frameon=False)
leg_meth.set_title("Inner Circle")

ax.text(-0.1,1.55,"Soil Carbon Sequestation\n73", ha="right")
ax.text(0.1,-1.75,"Forestry\n68", ha="center")
ax.text(1.55,-0.1,"Terrestrial CDR\n35", ha="left")
ax.text(1.4,0.6,"Marine CDR\n12", ha="left")
ax.text(1.,1.2,"CCS\n18", ha="left")
ax.text(0.1,1.55,"General/Other\n15", ha="left")

my_circle = plt.Circle( (0,0), 0.6, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)

### MRV topic - donut

In [ ]:
mrv_meth = df_MRVexplode.groupby("MRV_topic", as_index=False).size()
mrv_meth["col"] = mrv_meth.MRV_topic.replace(mrv_topic_colors)
# sorting
mrv_meth.MRV_topic = mrv_meth.MRV_topic.astype("category")
mrv_meth.MRV_topic = mrv_meth.MRV_topic.cat.set_categories(mrv_topic_colors.keys())
mrv_meth = mrv_meth.sort_values("MRV_topic")

In [ ]:
mrv_meth

In [ ]:
fig, ax = plt.subplots()

ax.pie(mrv_meth["size"], colors=mrv_meth["col"], startangle=90, radius=1.5)
my_circle = plt.Circle( (0,0), 0.7, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)

#ax.pie(cdr_meth["size"], colors=cdr_meth["col"], startangle=90, radius=1)

legend_mrv = []
for c in mrv_topic_colors.keys():
    legend_mrv.append(mpatches.Patch(color=mrv_topic_colors[c], label=c))
leg_mrv = plt.legend(handles=legend_mrv, bbox_to_anchor=(1.2,-0.1), fontsize=10, frameon=False,ncol=2)


ax.text(-0.1,1.55,"164", ha="right")
ax.text(1.07,-1.2,"36", ha="left")
ax.text(1.55,-0.1,"27", ha="left")
ax.text(1.3,0.85,"15", ha="left")
ax.text(0.9,1.25,"6", ha="left")
ax.text(0.65,1.39,"7", ha="left")
ax.text(0.43,1.47,"7", ha="left")
ax.text(0.1,1.55,"9", ha="left")


### Scientific method - donut

In [ ]:
df_resMethod = df_included.groupby('Primary research method',as_index=False).size()
df_resMethod["col"] = df_resMethod["Primary research method"].replace(resMethod_colors)
# sorting
df_resMethod["Primary research method"] = df_resMethod["Primary research method"].astype("category")
df_resMethod["Primary research method"] = df_resMethod["Primary research method"].cat.set_categories(resMethod_colors.keys())
df_resMethod = df_resMethod.sort_values("Primary research method")

In [ ]:
df_resMethod

In [ ]:
fig, ax = plt.subplots()

ax.pie(df_resMethod["size"], colors=df_resMethod["col"], startangle=90, radius=1.5)
my_circle = plt.Circle( (0,0), 0.7, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)

#ax.pie(cdr_meth["size"], colors=cdr_meth["col"], startangle=90, radius=1)

legend_res = []
for c in resMethod_colors.keys():
    if c == "Data analysis/Statistical analysis/Econometric":
        lab = "Data analysis/Statistical analysis/\nEconometric"
    else:
        lab = c
    
    legend_res.append(mpatches.Patch(color=resMethod_colors[c], label=lab))
leg_res = ax.legend(handles=legend_res, bbox_to_anchor=(1.7,-0.1), fontsize=10, frameon=False,ncol=2)

ax.text(-0.1,1.55,"52", ha="right")
ax.text(-1.6,-0.5,"19", ha="left")
ax.text(-0.9,-1.4,"37", ha="left")
ax.text(1,-1.2,"31", ha="left")
ax.text(1.55,0.1,"18", ha="left")
ax.text(1.2,1,"22", ha="left")
ax.text(0.3,1.52,"4", ha="left")
ax.text(0.05,1.55,"3", ha="left")


### Heatmap MRV vs.CDR

In [ ]:
cdrmrv = df_CDRexplode.explode(["MRV_focus","MRV_topic"])

# creating heatmap
hm_cdrmrv = cdrmrv.groupby(["CDR_group","MRV_topic"], as_index=False).size()
hm_cdrmrv = hm_cdrmrv.pivot(index="MRV_topic",columns="CDR_group",values="size")
hm_cdrmrv.fillna(0, inplace=True)
hm_cdrmrv_abs = hm_cdrmrv.copy()

# adding MRV counts
mrv_counts = hm_cdrmrv.sum(axis=1)
hm_cdrmrv["Total count"] = hm_cdrmrv.sum(axis=1)

# calculating CDR counts
cdr_counts = hm_cdrmrv.sum()
cdr_counts_df = {}
for i,c in enumerate(hm_cdrmrv.columns):
    cdr_counts_df[c] = [cdr_counts[i]]

# calculating relative numbers
hm_cdrmrv[hm_cdrmrv.columns[:-1]] = hm_cdrmrv[hm_cdrmrv.columns[:-1]].div(hm_cdrmrv["Total count"],axis=0)*100

# adding last row (absolute numbers of CDR counts)
hm_cdrmrv = pd.concat([hm_cdrmrv, pd.DataFrame(cdr_counts_df)])
hm_cdrmrv.index = hm_cdrmrv.index[:-1].tolist()+["Total count"]

In [ ]:
hm_cdrmrv_abs

In [ ]:
hm_cdrmrv_abs.iloc[2,0]

In [ ]:
import seaborn as sns
fig=plt.figure(figsize=(6, 6))

# define which areas are not used in the plot
mask = np.zeros_like(hm_cdrmrv)
mask[-1,:] = True
mask[:,-1] = True
mask[-1,-1] = True

ax = sns.heatmap(data=hm_cdrmrv, annot=False, cmap='Greys', mask=mask, 
                 linecolor='grey', linewidths=0.5, 
                 cbar=False, annot_kws={"fontsize":10}, fmt='.0f')


# add the total number in the total column 
for i in range(len(hm_cdrmrv.index)-1):
    ax.text(len(hm_cdrmrv.columns)-0.5, i+0.5, str(int(mrv_counts[i])), ha='center', va='center', color='black', fontsize =10)

# add the total number in the total row
for i in range(len(hm_cdrmrv.columns)-1):
    ax.text(i+0.5, len(hm_cdrmrv.columns)+1.5, str(int(cdr_counts[i])), ha='center', va='center', color='black', fontsize =10)
# add box on last column
ax.add_patch(plt.Rectangle((len(hm_cdrmrv.columns)-1, 0), 1, len(hm_cdrmrv.index), fill=None, ec='black', linewidth=2, alpha=1))
# add box on last row
ax.add_patch(plt.Rectangle((0, len(hm_cdrmrv.index)), len(hm_cdrmrv.columns), len(hm_cdrmrv.index), fill=None, ec='black', linewidth=2, alpha=1))

# add absolute numbers into cells
for c in range(len(hm_cdrmrv.columns)-1):
    for r in range(len(hm_cdrmrv.index)-1):
        if hm_cdrmrv_abs.iloc[r,c]==0 or hm_cdrmrv.iloc[r,c]>30:
            col = "white"
        else:
            col = "black"
        text = str(int(hm_cdrmrv_abs.iloc[r,c]))
        ax.text(c+0.5, r+0.5, text, ha='center', va='center', color=col, fontsize =10)


# Create legend and colors
cbar = ax.figure.colorbar(ax.collections[0])
_ = ax.tick_params(axis='x', labelbottom=False, labeltop=True, top=True, bottom=False)
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha='left', fontsize = 10)
_ = ax.set_yticklabels(ax.get_yticklabels(),fontsize = 10)
_ = plt.text(8.7,6.2, "Share of CDR method (%)\n        for each MRV topic", rotation=270)

fig.savefig('figures/mrv_cdr.png', bbox_inches="tight", facecolor='white', edgecolor='none')

### Heatmap MRV vs. scientific method

In [ ]:
# creating heatmap
hm_mrvres = df_MRVexplode.groupby(["Primary research method","MRV_topic"], as_index=False).size()
hm_mrvres = hm_mrvres.pivot(index="MRV_topic",columns="Primary research method",values="size")
hm_mrvres.fillna(0, inplace=True)
hm_mrvres_abs = hm_mrvres.copy()

# adding MRV counts
mrv_counts_r = hm_mrvres.sum(axis=1)
hm_mrvres["Total count"] = hm_mrvres.sum(axis=1)


# calculating research counts
res_counts = hm_mrvres.sum()
res_counts_df = {}
for i,c in enumerate(hm_mrvres.columns):
    res_counts_df[c] = [res_counts[i]]

# calculating relative numbers
hm_mrvres[hm_mrvres.columns[:-1]] = hm_mrvres[hm_mrvres.columns[:-1]].div(hm_mrvres["Total count"],axis=0)*100

# adding last row (absolute numbers of research counts)
hm_mrvres = pd.concat([hm_mrvres, pd.DataFrame(res_counts_df)])
hm_mrvres.index = hm_mrvres.index[:-1].tolist()+["Total count"]

In [ ]:
import seaborn as sns
from matplotlib import colormaps as cm
from matplotlib.ticker import LogFormatter 

fig=plt.figure(figsize=(6, 6))

# define which areas are not used in the plot
mask = np.zeros_like(hm_mrvres)
mask[-1,:] = True
mask[:,-1] = True
mask[-1,-1] = True

ax = sns.heatmap(data=hm_mrvres, annot=False, norm=mpl.colors.LogNorm(), cmap='Greys', mask=mask, 
                 linecolor='grey', linewidths=0.5, 
                 cbar=False, annot_kws={"fontsize":10}, fmt='.0f')


# add the total number in the total column 
for i in range(len(hm_mrvres.index)-1):
    ax.text(len(hm_mrvres.columns)-0.5, i+0.5, str(int(mrv_counts_r[i])), ha='center', va='center', color='black', fontsize =10)

# add the total number in the total row
for i in range(len(hm_mrvres.columns)-1):
    ax.text(i+0.5, len(hm_mrvres.columns)-.5, str(int(res_counts[i])), ha='center', va='center', color='black', fontsize =10)
# add box on last column
ax.add_patch(plt.Rectangle((len(hm_mrvres.columns)-1, 0), 1, len(hm_mrvres.index), fill=None, ec='black', linewidth=2, alpha=1))
# add box on last row
ax.add_patch(plt.Rectangle((0, len(hm_mrvres.index)), len(hm_mrvres.columns), len(hm_mrvres.index), fill=None, ec='black', linewidth=2, alpha=1))

# add absolute numbers into cells
for c in range(len(hm_mrvres.columns)-1):
    for r in range(len(hm_mrvres.index)-1):
        if hm_mrvres_abs.iloc[r,c]==0 or hm_mrvres.iloc[r,c]>6:
            col = "white"
        else:
            col = "black"
        text = str(int(hm_mrvres_abs.iloc[r,c]))
        ax.text(c+0.5, r+0.5, text, ha='center', va='center', color=col, fontsize =10)

# handle log-scale on colorbar
formatter = LogFormatter(2, labelOnlyBase=False)
mappable = ax.collections[0]
cb = plt.colorbar(mappable= mappable, ticks=[1,5,10,20,40,80], format=formatter)

# Create legend and colors
#cbar = ax.figure.colorbar(ax.collections[0], ticks = [1, 10, 30, 70])
_ = ax.tick_params(axis='x', labelbottom=False, labeltop=True, top=True, bottom=False)
_ = ax.set_xticklabels(ax.get_xticklabels(), rotation=30, ha='left', fontsize = 10)
_ = ax.set_yticklabels(ax.get_yticklabels(),fontsize = 10)
_ = plt.text(10.9,7.2, "                Share of \n primary research method (%)\n        for each MRV topic", rotation=270)

fig.savefig('figures/mrv_research.png', bbox_inches="tight", facecolor='white', edgecolor='none')

### All together

In [ ]:
fig = plt.figure(figsize=(9,14))
gs = fig.add_gridspec(2,7,height_ratios=[1.8,1],width_ratios=[1,0.7,0.2,0.5,0.5,1.1,1])

#### CDR methods #####################################################################
ax1 = fig.add_subplot(gs[0,0])
ax1.pie(cdr_group["size"], colors=cdr_group["col"], startangle=90, radius=1.5)
my_circle = plt.Circle( (0,0), 1.1, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)

ax1.pie(cdr_meth["size"], colors=cdr_meth["col"], startangle=90, radius=1)

legend_meth = []
for c in cdr_method_colors.keys():
    legend_meth.append(mpatches.Patch(color=cdr_method_colors[c], label=cdr_method_label[c]))
leg_meth = plt.legend(handles=legend_meth, bbox_to_anchor=(2,-0.4), fontsize=8, frameon=False,ncol=2)

ax1.text(-2.7,-2.4,"Inner Circle")

ax1.text(-0.1,1.55,"Soil Carbon Sequestation\n73", ha="right")
ax1.text(0.1,-2.2,"Forestry\n68", ha="center")
ax1.text(1.55,-0.2,"Terrestrial CDR\n35", ha="left")
ax1.text(1.4,0.6,"Marine CDR\n12", ha="left")
ax1.text(1.,1.2,"CCS\n18", ha="left")
ax1.text(0.1,1.55,"General/Other\n15", ha="left")

my_circle = plt.Circle( (0,0), 0.6, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)


#### MRV topics ########################################################################
ax2 = fig.add_subplot(gs[0,3:5])

ax2.pie(mrv_meth["size"], colors=mrv_meth["col"], startangle=90, radius=1.4)
my_circle = plt.Circle( (0,0), 0.7, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)

legend_mrv = []
for c in mrv_topic_colors.keys():
    legend_mrv.append(mpatches.Patch(color=mrv_topic_colors[c], label=c))
leg_mrv = plt.legend(handles=legend_mrv, bbox_to_anchor=(1.2,-0.3), fontsize=8, frameon=False,ncol=1)

ax2.text(-0.1,1.45,"164", ha="right")
ax2.text(1.07,-1.2,"36", ha="left")
ax2.text(1.45,0.1,"27", ha="left")
ax2.text(1.2,0.9,"15", ha="left")
ax2.text(0.9,1.25,"6", ha="left")
ax2.text(0.67,1.29,"7", ha="left")
ax2.text(0.45,1.37,"7", ha="left")
ax2.text(0.1,1.45,"9", ha="left")

### Scientific methods #################################################################
ax3 = fig.add_subplot(gs[0,6:])

ax3.pie(df_resMethod["size"], colors=df_resMethod["col"], startangle=90, radius=1.5)
my_circle = plt.Circle( (0,0), 0.7, color='white')
p=plt.gcf()
p.gca().add_artist(my_circle)

#ax.pie(cdr_meth["size"], colors=cdr_meth["col"], startangle=90, radius=1)

legend_res = []
for c in resMethod_colors.keys():
    if c == "Data analysis/Statistical analysis/Econometric":
        lab = "Data analysis/Statistical analysis/\nEconometric"
    else:
        lab = c
    
    legend_res.append(mpatches.Patch(color=resMethod_colors[c], label=lab))
leg_res = ax3.legend(handles=legend_res, bbox_to_anchor=(1.7,-0.4), fontsize=8, frameon=False,ncol=1)

ax3.text(-0.1,1.55,"52", ha="right")
ax3.text(-1.8,-0.6,"19", ha="left")
ax3.text(-0.9,-1.6,"37", ha="left")
ax3.text(1,-1.2,"31", ha="left")
ax3.text(1.55,0.1,"18", ha="left")
ax3.text(1.2,1,"22", ha="left")
ax3.text(0.25,1.52,"4", ha="left")
ax3.text(0.0,1.55,"3", ha="left")

##### MRV vs CDR
ax4 = fig.add_subplot(gs[1,0:3])

# define which areas are not used in the plot (=True)
mask = np.zeros_like(hm_cdrmrv)
mask[-1,:] = True
mask[:,-1] = True
mask[-1,-1] = True

ax4 = sns.heatmap(data=hm_cdrmrv, annot=False, cmap='Greys', mask=mask, 
                 linecolor='grey', linewidths=0.5, 
                 cbar=False, annot_kws={"fontsize":8}, fmt='.0f')


# add the total number in the total column 
for i in range(len(hm_cdrmrv.index)-1):
    ax4.text(len(hm_cdrmrv.columns)-0.5, i+0.5, str(int(mrv_counts[i])), ha='center', va='center', color='black', fontsize =8)

# add the total number in the total row
for i in range(len(hm_cdrmrv.columns)-1):
    ax4.text(i+0.5, len(hm_cdrmrv.columns)+1.5, str(int(cdr_counts[i])), ha='center', va='center', color='black', fontsize =8)
# add box on last column
ax4.add_patch(plt.Rectangle((len(hm_cdrmrv.columns)-1, 0), 1, len(hm_cdrmrv.index), fill=None, ec='black', linewidth=2, alpha=1))
# add box on last row
ax4.add_patch(plt.Rectangle((0, len(hm_cdrmrv.index)), len(hm_cdrmrv.columns), len(hm_cdrmrv.index), fill=None, ec='black', linewidth=2, alpha=1))

# add absolute numbers into cells
for c in range(len(hm_cdrmrv.columns)-1):
    for r in range(len(hm_cdrmrv.index)-1):
        if hm_cdrmrv_abs.iloc[r,c]==0 or hm_cdrmrv.iloc[r,c]>30:
            col = "white"
        else:
            col = "black"
        text = str(int(hm_cdrmrv_abs.iloc[r,c]))
        ax4.text(c+0.5, r+0.5, text, ha='center', va='center', color=col, fontsize=10)


# Create legend and colors
cbar = ax4.figure.colorbar(ax4.collections[0])
_ = ax4.tick_params(axis='x', labelbottom=False, labeltop=True, top=True, bottom=False)
_ = ax4.set_xticklabels(ax4.get_xticklabels(), rotation=30, ha='left', fontsize = 8)
_ = ax4.set_yticklabels(ax4.get_yticklabels(),fontsize = 8)
_ = plt.text(9.4,6.2, "Share of CDR method (%)\n        for each MRV topic", rotation=270)

##### MRV vs CDR
ax5 = fig.add_subplot(gs[1,5:])

# define which areas are not used in the plot
mask = np.zeros_like(hm_mrvres)
mask[-1,:] = True
mask[:,-1] = True
mask[-1,-1] = True

ax5 = sns.heatmap(data=hm_mrvres, annot=False, norm=mpl.colors.LogNorm(), cmap='Greys', mask=mask, 
                 linecolor='grey', linewidths=0.5, 
                 cbar=False, annot_kws={"fontsize":10}, fmt='.0f')


# add the total number in the total column 
for i in range(len(hm_mrvres.index)-1):
    ax5.text(len(hm_mrvres.columns)-0.5, i+0.5, str(int(mrv_counts_r[i])), ha='center', va='center', color='black', fontsize =8)

# add the total number in the total row
for i in range(len(hm_mrvres.columns)-1):
    ax5.text(i+0.5, len(hm_mrvres.columns)-.5, str(int(res_counts[i])), ha='center', va='center', color='black', fontsize =8)
# add box on last column
ax5.add_patch(plt.Rectangle((len(hm_mrvres.columns)-1, 0), 1, len(hm_mrvres.index), fill=None, ec='black', linewidth=2, alpha=1))
# add box on last row
ax5.add_patch(plt.Rectangle((0, len(hm_mrvres.index)), len(hm_mrvres.columns), len(hm_mrvres.index), fill=None, ec='black', linewidth=2, alpha=1))

# add absolute numbers into cells
for c in range(len(hm_mrvres.columns)-1):
    for r in range(len(hm_mrvres.index)-1):
        if hm_mrvres_abs.iloc[r,c]==0 or hm_mrvres.iloc[r,c]>6:
            col = "white"
        else:
            col = "black"
        text = str(int(hm_mrvres_abs.iloc[r,c]))
        ax5.text(c+0.5, r+0.5, text, ha='center', va='center', color=col, fontsize = 10)

# handle log-scale on colorbar
formatter = LogFormatter(2, labelOnlyBase=False)
mappable = ax5.collections[0]
cb = plt.colorbar(mappable= mappable, ticks=[1,5,10,20,40,80], format=formatter)

# Create legend and colors
#cbar = ax.figure.colorbar(ax.collections[0], ticks = [1, 10, 30, 70])
_ = ax5.tick_params(axis='x', labelbottom=False, labeltop=True, top=True, bottom=False)
_ = ax5.set_xticklabels(ax5.get_xticklabels(), rotation=30, ha='left', fontsize = 8)
_ = ax5.set_yticklabels(ax5.get_yticklabels(),fontsize = 8)
_ = ax5.text(11.6,7.2, "                Share of \n primary research method (%)\n        for each MRV topic", rotation=270)

plt.text(-18, -13.6, "CDR Methods", fontsize=12)
plt.text(-5.5, -13.6, "MRV Topics", fontsize=12)
plt.text(6, -13.6, "Research Methods", fontsize=12)

plt.text(-20, -13.55, "A", fontsize=15)
plt.text(-7.5, -13.55, "B", fontsize=15)
plt.text(4, -13.55, "C", fontsize=15)
plt.text(-20, -0.3, "D", fontsize=15)
plt.text(-2, -0.3, "E", fontsize=15)

fig.savefig('figures/fig2.png', bbox_inches="tight", facecolor='white', edgecolor='none')